# A scratch pad to run model inference manually


In [1]:

import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('ggplot')

from typing import Optional, List, Dict, Union

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import Tensor
from torch import optim
from torch.utils.data import random_split, DataLoader, TensorDataset

from pathlib import Path
import transformers


from loguru import logger
logger.add(os.sys.stderr, format="{time} {level} {message}", level="INFO")


/media/wassname/SGIronWolf/projects5/elk/discovering_latent_knowledge/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1

In [2]:
# load my code
%load_ext autoreload
%autoreload 2


from src.extraction.config import ExtractConfig
from src.prompts.prompt_loading import load_preproc_dataset
from src.models.load import load_model
from src.datasets.intervene import create_cache_interventions 
from src.prompts.prompt_loading import load_prompt_structure
from src.repe import repe_pipeline_registry
repe_pipeline_registry()


CUDA extension not installed.
CUDA extension not installed.


In [3]:
# config transformers
from datasets import set_caching_enabled, disable_caching
disable_caching()

os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # cache busting for the transformers map and ds steps
# !rm -rf ~/.cache/huggingface/datasets/generator


## Load model

In [4]:
ds_name='amazon_polarity'
cfg = ExtractConfig(max_examples=(40, 40),
                    intervention_fit_examples=10,
                    )
print(cfg)
batch_size = cfg.batch_size

model, tokenizer = load_model(cfg.model, pad_token_id=cfg.pad_token_id)
print(model)

N_train, N_test = cfg.max_examples
N=sum(cfg.max_examples)
ds_tokens = load_preproc_dataset(ds_name, tokenizer, N=N, seed=cfg.seed, num_shots=cfg.num_shots, max_length=cfg.max_length, prompt_format=cfg.prompt_format)


2023-12-16 10:14:43.517 | INFO     | src.models.load:verbose_change_param:24 - changing use_cache from True to False
2023-12-16T10:14:43.517976+0800 INFO changing use_cache from True to False
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2023-12-16 10:14:43.853 | INFO     | src.models.load:verbose_change_param:24 - changing pad_token_id from None to 50256
2023-12-16T10:14:43.853595+0800 INFO changing pad_token_id from None to 50256
2023-12-16 10:14:43.854 | INFO     | src.models.load:verbose_change_param:24 - changing padding_side from right to left
2023-12-16T10:14:43.854275+0800 INFO changing padding_side from right to left
2023-12-16 10:14:43.854 | INFO     | src.models.load:verbose_change_param:24 - changing truncation_side from right to left
2023-12-16T10:14:43.854771+0800 INFO changing truncation_side from right to left
Generating train split: 0 examples [00:00, ? examples/s]

Extracting 11 variants of each prompt


Generating train split: 242 examples [00:40,  5.91 examples/s]
choice_ids: 100%|██████████| 242/242 [00:00<00:00, 6967.33 examples/s]
2023-12-16 10:15:28.371 | INFO     | src.prompts.prompt_loading:load_preproc_dataset:368 - truncation rate: 0.0 on amazon_polarity
2023-12-16T10:15:28.371476+0800 INFO truncation rate: 0.0 on amazon_polarity


median token length: 440.0 for amazon_polarity. max_length=1000


Filter: 100%|██████████| 242/242 [00:00<00:00, 2118.60 examples/s]

num_rows (after filtering out truncated rows) 242=>242


In [ ]:
honesty_rep_reader = create_cache_interventions(model, tokenizer, cfg)


## Generate
